In [99]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import itertools
import numpy as np
import sklearn as  sk
from scipy import stats
from statsmodels.graphics.mosaicplot import mosaic
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier,VotingClassifier
from sklearn import svm
import xgboost as xgb

from sklearn.model_selection import cross_val_score,KFold,train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix
from subprocess import check_output

In [100]:
train_df = pd.read_csv('data/churn_train.csv')
test_df = pd.read_csv('data/churn_test.csv')
df = pd.read_csv('data/churn.csv')

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
avg_dist                  50000 non-null float64
avg_rating_by_driver      49799 non-null float64
avg_rating_of_driver      41878 non-null float64
avg_surge                 50000 non-null float64
city                      50000 non-null object
last_trip_date            50000 non-null object
phone                     49604 non-null object
signup_date               50000 non-null object
surge_pct                 50000 non-null float64
trips_in_first_30_days    50000 non-null int64
luxury_car_user           50000 non-null bool
weekday_pct               50000 non-null float64
dtypes: bool(1), float64(6), int64(1), object(4)
memory usage: 4.2+ MB


In [102]:
df.head()

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,last_trip_date,phone,signup_date,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct
0,3.67,5.0,4.7,1.10,King's Landing,2014-06-17,iPhone,2014-01-25,15.4,4,True,46.2
1,8.26,5.0,5.0,1.00,Astapor,2014-05-05,Android,2014-01-29,0.0,0,False,50.0
2,0.77,5.0,4.3,1.00,Astapor,2014-01-07,iPhone,2014-01-06,0.0,3,False,100.0
3,2.36,4.9,4.6,1.14,King's Landing,2014-06-29,iPhone,2014-01-10,20.0,9,True,80.0
4,3.13,4.9,4.4,1.19,Winterfell,2014-03-15,Android,2014-01-27,11.8,14,False,82.4


In [103]:
df.isnull().sum()

avg_dist                     0
avg_rating_by_driver       201
avg_rating_of_driver      8122
avg_surge                    0
city                         0
last_trip_date               0
phone                      396
signup_date                  0
surge_pct                    0
trips_in_first_30_days       0
luxury_car_user              0
weekday_pct                  0
dtype: int64

In [104]:
def clean_df(df,datetime='%Y-%m-%d',use_datetime=False):
    for column in df.columns:
        try:
            df[column] = df[column].astype(float)
        except:
            try:
                if use_datetime==True:
                    df[column] = pd.to_datetime(df[column],format=datetime)
            except:
                print "Couldn't convert columns: ", column

In [105]:
clean_df(df,use_datetime=True)

Couldn't convert columns:  city
Couldn't convert columns:  phone


In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
avg_dist                  50000 non-null float64
avg_rating_by_driver      49799 non-null float64
avg_rating_of_driver      41878 non-null float64
avg_surge                 50000 non-null float64
city                      50000 non-null object
last_trip_date            50000 non-null datetime64[ns]
phone                     49604 non-null object
signup_date               50000 non-null datetime64[ns]
surge_pct                 50000 non-null float64
trips_in_first_30_days    50000 non-null float64
luxury_car_user           50000 non-null float64
weekday_pct               50000 non-null float64
dtypes: datetime64[ns](2), float64(8), object(2)
memory usage: 4.6+ MB


In [107]:
df.head()

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,last_trip_date,phone,signup_date,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct
0,3.67,5.0,4.7,1.10,King's Landing,2014-06-17,iPhone,2014-01-25,15.4,4.0,1.0,46.2
1,8.26,5.0,5.0,1.00,Astapor,2014-05-05,Android,2014-01-29,0.0,0.0,0.0,50.0
2,0.77,5.0,4.3,1.00,Astapor,2014-01-07,iPhone,2014-01-06,0.0,3.0,0.0,100.0
3,2.36,4.9,4.6,1.14,King's Landing,2014-06-29,iPhone,2014-01-10,20.0,9.0,1.0,80.0
4,3.13,4.9,4.4,1.19,Winterfell,2014-03-15,Android,2014-01-27,11.8,14.0,0.0,82.4


In [108]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
avg_dist,50000.0,5.796827,5.707357,0.0,2.42,3.88,6.94,160.96
avg_rating_by_driver,49799.0,4.778158,0.446652,1.0,4.70,5.00,5.00,5.00
avg_rating_of_driver,41878.0,4.601559,0.617338,1.0,4.30,4.90,5.00,5.00
avg_surge,50000.0,1.074764,0.222336,1.0,1.00,1.00,1.05,8.00
surge_pct,50000.0,8.849536,19.958811,0.0,0.00,0.00,8.60,100.00
trips_in_first_30_days,50000.0,2.278200,3.792684,0.0,0.00,1.00,3.00,125.00
luxury_car_user,50000.0,0.377080,0.484660,0.0,0.00,0.00,1.00,1.00
weekday_pct,50000.0,60.926084,37.081503,0.0,33.30,66.70,100.00,100.00


In [109]:
df.isnull().sum()

avg_dist                     0
avg_rating_by_driver       201
avg_rating_of_driver      8122
avg_surge                    0
city                         0
last_trip_date               0
phone                      396
signup_date                  0
surge_pct                    0
trips_in_first_30_days       0
luxury_car_user              0
weekday_pct                  0
dtype: int64

In [110]:
df.loc[:,'avg_rating_of_driver'][df['avg_rating_of_driver'].isnull()]=df['avg_rating_by_driver'][df['avg_rating_of_driver'].isnull()]

/home/aamir/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [111]:
df['avg_rating_of_driver'].isnull().sum()

67

In [112]:
df['avg_rating_by_driver'].isnull().sum()

201

In [113]:
df.loc[:,'phone'][df.loc[:,'phone'].isnull()] = 0

/home/aamir/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [114]:
df.isnull().sum()

avg_dist                    0
avg_rating_by_driver      201
avg_rating_of_driver       67
avg_surge                   0
city                        0
last_trip_date              0
phone                       0
signup_date                 0
surge_pct                   0
trips_in_first_30_days      0
luxury_car_user             0
weekday_pct                 0
dtype: int64

In [115]:
df.mean()

avg_dist                   5.796827
avg_rating_by_driver       4.778158
avg_rating_of_driver       4.631310
avg_surge                  1.074764
surge_pct                  8.849536
trips_in_first_30_days     2.278200
luxury_car_user            0.377080
weekday_pct               60.926084
dtype: float64

In [116]:
df = df.fillna(df.mean())

In [117]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
avg_dist                  50000 non-null float64
avg_rating_by_driver      50000 non-null float64
avg_rating_of_driver      50000 non-null float64
avg_surge                 50000 non-null float64
city                      50000 non-null object
last_trip_date            50000 non-null datetime64[ns]
phone                     50000 non-null object
signup_date               50000 non-null datetime64[ns]
surge_pct                 50000 non-null float64
trips_in_first_30_days    50000 non-null float64
luxury_car_user           50000 non-null float64
weekday_pct               50000 non-null float64
dtypes: datetime64[ns](2), float64(8), object(2)
memory usage: 4.6+ MB


In [118]:
df['last_trip_date']=(df['last_trip_date'] < datetime.datetime(2014,6,1)).astype(int)

In [119]:
X = df.drop(['last_trip_date'],1)

In [120]:
y = df['last_trip_date']

In [121]:
y.value_counts()

1    31196
0    18804
Name: last_trip_date, dtype: int64

In [122]:
X.head()

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,phone,signup_date,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct
0,3.67,5.0,4.7,1.10,King's Landing,iPhone,2014-01-25,15.4,4.0,1.0,46.2
1,8.26,5.0,5.0,1.00,Astapor,Android,2014-01-29,0.0,0.0,0.0,50.0
2,0.77,5.0,4.3,1.00,Astapor,iPhone,2014-01-06,0.0,3.0,0.0,100.0
3,2.36,4.9,4.6,1.14,King's Landing,iPhone,2014-01-10,20.0,9.0,1.0,80.0
4,3.13,4.9,4.4,1.19,Winterfell,Android,2014-01-27,11.8,14.0,0.0,82.4


In [123]:
X.drop(['avg_dist','signup_date'],axis=1,inplace=True)

In [124]:
X.head()

,avg_rating_by_driver,avg_rating_of_driver,avg_surge,city,phone,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct
0,5.0,4.7,1.10,King's Landing,iPhone,15.4,4.0,1.0,46.2
1,5.0,5.0,1.00,Astapor,Android,0.0,0.0,0.0,50.0
2,5.0,4.3,1.00,Astapor,iPhone,0.0,3.0,0.0,100.0
3,4.9,4.6,1.14,King's Landing,iPhone,20.0,9.0,1.0,80.0
4,4.9,4.4,1.19,Winterfell,Android,11.8,14.0,0.0,82.4


In [125]:
X_dummy = pd.get_dummies(X,columns=['city','phone'])
X_dummy.head()

,avg_rating_by_driver,avg_rating_of_driver,avg_surge,surge_pct,trips_in_first_30_days,luxury_car_user,weekday_pct,city_Astapor,city_King's Landing,city_Winterfell,phone_0,phone_Android,phone_iPhone
0,5.0,4.7,1.10,15.4,4.0,1.0,46.2,0,1,0,0,0,1
1,5.0,5.0,1.00,0.0,0.0,0.0,50.0,1,0,0,0,1,0
2,5.0,4.3,1.00,0.0,3.0,0.0,100.0,1,0,0,0,0,1
3,4.9,4.6,1.14,20.0,9.0,1.0,80.0,0,1,0,0,0,1
4,4.9,4.4,1.19,11.8,14.0,0.0,82.4,0,0,1,0,1,0


In [126]:
X_dummy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 13 columns):
avg_rating_by_driver      50000 non-null float64
avg_rating_of_driver      50000 non-null float64
avg_surge                 50000 non-null float64
surge_pct                 50000 non-null float64
trips_in_first_30_days    50000 non-null float64
luxury_car_user           50000 non-null float64
weekday_pct               50000 non-null float64
city_Astapor              50000 non-null uint8
city_King's Landing       50000 non-null uint8
city_Winterfell           50000 non-null uint8
phone_0                   50000 non-null uint8
phone_Android             50000 non-null uint8
phone_iPhone              50000 non-null uint8
dtypes: float64(7), uint8(6)
memory usage: 3.0 MB
